In [1]:
import pandas as pd

In [2]:
def get_count_rows(filename):
    res = pd.read_csv(filename, chunksize=(10 ** 6))
    count_rows = 0
    for chunk in res:
        count_rows += len(chunk.index)
    return count_rows

In [3]:
def join_label_frequency(labels1, labels2):
    result = {key: labels1[key] for key in labels1.keys()}
    for key in labels2.keys():
        if key not in result:
            result[key] = labels2[key]
        else:
            result[key] += labels2[key]
    return result

In [4]:
def get_label_frequency(filename):
    labels = {}
    res = pd.read_csv(filename, chunksize=(10 ** 6))
    for chunk in res:
        chunk_labels = chunk[' Label'].value_counts().to_dict()
        labels = join_label_frequency(labels, chunk_labels)
    return labels

In [5]:
labels = get_label_frequency('../dataset/03-11/LDAP.csv')
labels = join_label_frequency(labels, get_label_frequency('../dataset/03-11/MSSQL.csv'))
labels = join_label_frequency(labels, get_label_frequency('../dataset/03-11/NetBIOS.csv'))
labels = join_label_frequency(labels, get_label_frequency('../dataset/03-11/Portmap.csv'))
labels = join_label_frequency(labels, get_label_frequency('../dataset/03-11/Syn.csv'))
labels = join_label_frequency(labels, get_label_frequency('../dataset/03-11/UDP.csv'))
labels = join_label_frequency(labels, get_label_frequency('../dataset/03-11/UDPLag.csv'))
print(labels)

/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/2616902257.py:4: DtypeWarning: C

{'LDAP': 1915122, 'NetBIOS': 3657497, 'BENIGN': 56965, 'MSSQL': 5787453, 'Portmap': 186960, 'Syn': 4891500, 'UDP': 3867155, 'UDPLag': 1873}


In [6]:
count_rows = get_count_rows('../dataset/03-11/LDAP.csv')
count_rows += get_count_rows('../dataset/03-11/MSSQL.csv')
count_rows += get_count_rows('../dataset/03-11/NetBIOS.csv')
count_rows += get_count_rows('../dataset/03-11/Portmap.csv')
count_rows += get_count_rows('../dataset/03-11/Syn.csv')
count_rows += get_count_rows('../dataset/03-11/UDP.csv')
count_rows += get_count_rows('../dataset/03-11/UDPLag.csv')

/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_6669/893618011.py:4: DtypeWarning: Columns 

In [7]:
print(count_rows)

20364525


In [8]:
import json
result = {
    "labels": labels,
    "total": count_rows
}
with open("label_frequency.json", "w") as f:
    json.dump(result, f, indent=4)

In [5]:
labels = get_label_frequency('../dataset/clean-sample.csv')
print(labels)

/tmp/ipykernel_9867/2616902257.py:4: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:
/tmp/ipykernel_9867/2616902257.py:4: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in res:


{'MSSQL': 568391, 'NetBIOS': 359204, 'LDAP': 188088, 'BENIGN': 5615, 'Syn': 480392, 'UDP': 379799, 'Portmap': 18362, 'UDPLag': 184}
